## Python code

### Imports

In [114]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script for Thymio and local occupancy
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

In [115]:
robot = Thymio.serial(port='COM3', refreshing_rate=0.1)

### Accessible functions for the robot

In [116]:
#dir(robot)

### Read/Write variables

In [117]:
#variables = robot.variable_description()
#for var in variables :
#    print(var)

### Variables globales

In [138]:
PROXIMITY_THRESHOLD = 3800 #around 2cm
TIME_SLEEP_OBSTACLE = 0.1 #in seconds
TIME_SLEEP_DATA = 0.5 # 0.2 = 5Hz frequency
MOTOR_SPEED = 80 #both motor are set with this value (one in each direction)

SENSOR_0 = 0
SENSOR_1 = 1
SENSOR_2 = 2
SENSOR_3 = 3
SENSOR_4 = 4
SENSOR_5 = 5
SENSOR_6 = 6
ALL_FRONT_SENSOR = 5
PARTIALY_ALL_FRONT_SENSOR = 4
COMPTEUR = 6
DIST_WHEELS = 9.4

### Main functions

If/elif

Initialisations

In [174]:
def set_course(target, x, y, alpha):
    
    dy = target[1] - y
    dx = target[0] - x
    alphatarg = np.arctan2(dy,dx)
    dalpha = alphatarg - alpha
    
    print('delta ',dx,dy,dalpha)
    
    if  dalpha > 0.05 :
        robot.set_var("motor.left.target", int(2**16)-int(MOTOR_SPEED*dalpha))
        robot.set_var("motor.right.target", int(MOTOR_SPEED*dalpha)) 
    
    elif dalpha < -0.05:
        robot.set_var("motor.left.target", int(MOTOR_SPEED*(-dalpha)))
        robot.set_var("motor.right.target", int(2**16)-int(MOTOR_SPEED*(-dalpha)))
    
    else:
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", MOTOR_SPEED) 
    return

In [177]:
def go_to_goal(target_list):
    t0 = time.time()
    t1 = time.time()
    x = y = alpha = 0
    target_list = [[0,0],[10,10],[20,20],[30,30],[40,40]]
    target_reached =0
    last_target_reached =0
    follow_obstacle = 0
    iteration = 0
    id_target = 1
    
    while (not last_target_reached) : #simulation of big while loop
        target = target_list[id_target]
        goal = target_list[len(target_list)-1]
        t0 = time.time()
        
        print('target = ', target)

        if follow_obstacle == 1 :
            robot.set_var("motor.left.target", MOTOR_SPEED)        
            robot.set_var("motor.right.target", MOTOR_SPEED)       
        else :
            set_course(target, x, y, alpha)

        sensor_measurment = robot["prox.horizontal"]

        if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
            pass
        else:
            obstacle_direction = np.nonzero(sensor_measurment)[0]
            while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):
                #robot.set_var('leds.top', 123)

                timer = t1-t0
                t0 = time.time()

                lspeed = robot.get_var("motor.left.target")
                rspeed = robot.get_var("motor.right.target")

                if (lspeed > 100):
                    lspeed = lspeed - int(2**16)
                    d = (-lspeed+rspeed)*timer/45
                    alpha = alpha + np.arcsin(2*d/DIST_WHEELS)

                elif (rspeed > 100):
                    rspeed = rspeed - int(2**16)
                    d = (lspeed-rspeed)*timer/45
                    alpha = alpha - np.arcsin(2*d/DIST_WHEELS)
                else:
                    d = (lspeed+rspeed)*timer/45
                    x=x+d*np.cos(alpha)
                    y=y+d*np.sin(alpha)

                print('paramètres ',timer, lspeed, rspeed, d)

                print('robot = ',x,y,alpha)

                if (lspeed > 100):
                    alpha = alpha + np.arcsin(2*d/DIST_WHEELS)
                elif (rspeed > 100):
                    alpha = alpha - np.arcsin(2*d/DIST_WHEELS)
                else:
                    x=x+d*np.cos(alpha)
                    y=y+d*np.sin(alpha)

                if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
                    if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
                        robot.set_var("motor.left.target", MOTOR_SPEED)
                        robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                    else:
                        robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                        robot.set_var("motor.right.target", MOTOR_SPEED)
                else:
                    if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
                        robot.set_var("motor.left.target", MOTOR_SPEED)
                        robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                    elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
                        robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                        robot.set_var("motor.right.target", MOTOR_SPEED)
                    elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
                        robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                        robot.set_var("motor.right.target", MOTOR_SPEED)

                time.sleep(TIME_SLEEP_OBSTACLE)

                sensor_measurment = robot["prox.horizontal"]
                obstacle_direction = np.nonzero(sensor_measurment)[0]
                t1 = time.time()
            id_target = id_target + 1
            if id_target>len(target_list)-1:
                id_target = len(target_list)-1
            
        if (sensor_measurment[0] > 1000) or (sensor_measurment[4] > 1000) :
            follow_obstacle = 1
            iteration = 0
        else : 
            iteration = iteration + 1
            if iteration == COMPTEUR :
                follow_obstacle = 0
                

        time.sleep(TIME_SLEEP_DATA)
        t1 = time.time()
        timer = t1-t0
        

        lspeed = robot.get_var("motor.left.target")
        rspeed = robot.get_var("motor.right.target")

        if (lspeed > 100):
            lspeed = lspeed - int(2**16)
            d = (-lspeed+rspeed)*timer/45
            alpha = alpha + np.arcsin(2*d/DIST_WHEELS)

        elif (rspeed > 100):
            rspeed = rspeed - int(2**16)
            d = (lspeed-rspeed)*timer/45
            alpha = alpha - np.arcsin(2*d/DIST_WHEELS)
        else:
            d = (lspeed+rspeed)*timer/45
            x=x+d*np.cos(alpha)
            y=y+d*np.sin(alpha)


        print('paramètres ',timer, lspeed, rspeed, d)

        print('robot = ',x,y,alpha)

        if -0.75 < x-target[0] < 0.75 and -0.75 < y-target[1] < 0.75 :
            target_reached = 1
            print('coucou')
            id_target = id_target+1
            if id_target>len(target_list)-1:
                id_target = len(target_list)-1
            
        if -1 < x-goal[0] < 1 and -1 < y-goal[1] < 1 :
            last_target_reached = 1
            robot.set_var("motor.left.target", 0)
            robot.set_var("motor.right.target", 0)

    return

In [178]:
go_to_goal(target_list)

target =  [10, 10]
delta  10 10 0.7853981633974483
paramètres  0.5176522731781006 -62 62 1.4264195972018772
robot =  0 0 0.3083569894761109
target =  [10, 10]
delta  10 10 0.4770411739213374
paramètres  0.5102248191833496 -38 38 0.8617130279541015
robot =  0 0 0.4927432146963421
target =  [10, 10]
delta  10 10 0.2926549487011062
paramètres  0.5080347061157227 -23 23 0.5193243662516276
robot =  0 0 0.6034638438484354
target =  [10, 10]
delta  10 10 0.18193431954901285
paramètres  0.505272626876831 -14 14 0.31439185672336156
robot =  0 0 0.6704057138636197
target =  [10, 10]
delta  10 10 0.11499244953382859
paramètres  0.5065832138061523 -9 9 0.20263328552246093
robot =  0 0 0.7135325463947504
target =  [10, 10]
delta  10 10 0.07186561700269789
paramètres  0.5052108764648438 -5 5 0.11226908365885417
robot =  0 0 0.7374218576747107
target =  [10, 10]
delta  10 10 0.04797630572273759
paramètres  0.506037712097168 80 80 1.7992451985677083
robot =  1.3318094155713265 1.209779717619912 0.7374

<ipython-input-177-2f6b2b8f9c04>:111: RuntimeWarning: invalid value encountered in arcsin
  alpha = alpha + np.arcsin(2*d/DIST_WHEELS)


paramètres  0.5076444149017334 -65357 65357 1474.5829344325596
robot =  2.2727584429764676 21.823312084603405 nan
target =  [20, 20]
delta  17.727241557023532 -1.8233120846034048 nan
paramètres  0.5066378116607666 80 80 1.801378885904948
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.508948564529419 80 80 1.8095948961046007
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5056755542755127 80 80 1.797957526312934
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5049247741699219 80 80 1.7952880859375
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5099954605102539 80 80 1.8133171929253473
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.504751443862915 80 80 1.7946718004014757
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.5167465209960938 80 80 1.8373209635416667
robot =  nan nan nan
target =  [20, 20]
delta  nan nan nan
paramètres  0.50340

KeyboardInterrupt: 

### Tests

ANN

In [38]:
WEIGHT_L=[40,25,-20,-25,-40, 0,0] #sensor_5 = 30 ; sensor_6 = -10 (here = 0 beacause of the cable)
WEIGHT_R=[-40,-25,-20,25,40, 0,0] #sensor_5 = -10 ; sensor_6 = 30 (here = 0 beacause of the cable)

SENSOR_SCALE = 4000 #used for rescaling the data send by the sensor to give it to the motors
THRESHOLD = 20
MOTOR_SPEED = 80 #both motor are set with this value (one in each direction)

for i in range(10000000): #simulation of big while loop 
    
    robot.set_var("motor.left.target", int(2**16) - MOTOR_SPEED)        #TO REMOVE
    robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    #print(sensor_measurment)
    
    measurement_rescaled = np.divide(sensor_measurment, SENSOR_SCALE)
    #print(measurement_rescaled)
    weighted_measurment_l = np.multiply(measurement_rescaled, WEIGHT_L)
    #print(weighted_measurment_l)
    weighted_measurment_r = np.multiply(measurement_rescaled, WEIGHT_R)
    #print(weighted_measurment_r)

    motor_speed_l = np.sum(weighted_measurment_l)
    motor_speed_r = np.sum(weighted_measurment_r)

    #print(motor_speed_l)
    #print(motor_speed_r)
    
    if np.any(np.greater(motor_speed_l, THRESHOLD)) or np.any(np.greater(motor_speed_r, THRESHOLD)) or np.any(np.less(motor_speed_l, -THRESHOLD)) or np.any(np.less(motor_speed_r, -THRESHOLD)):       
        
        obstacle_direction = np.nonzero(sensor_measurment)[0]
        
        if (np.any(np.equal(obstacle_direction,SENSOR_2))) and (np.shape(obstacle_direction)[0]==1):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
        else :
            if motor_speed_l<0:
                #print(2**16 + int(motor_speed_l))
                robot.set_var("motor.left.target", int(2**16) + int(motor_speed_l)-1)
            else :
                robot.set_var("motor.left.target", int(motor_speed_l))

            if motor_speed_r<0:
                #print(2**16 + int(motor_speed_r))
                robot.set_var("motor.right.target", int(2**16) + int(motor_speed_r)-1)
            else :
                robot.set_var("motor.right.target", int(motor_speed_r))
        
        
        """
        robot.set_var("motor.left.target", 0)
        robot.set_var("motor.right.target", 0)
        
        robot.set_var("motor.left.target", 2**16-motor_speed_l)
        robot.set_var("motor.right.target", 2**16-motor_speed_r)
        """
    
    time.sleep(TIME_SLEEP_OBSTACLE)

    

KeyboardInterrupt: 

In [179]:
sensor_measurment = robot["prox.horizontal"]
obstacle_direction = np.nonzero(sensor_measurment)[0]

print(sensor_measurment)
print(obstacle_direction)
#print(np.argmax(sensor_measurment))
print((np.shape(obstacle_direction)[0] == 1) and (np.any(np.equal(obstacle_direction,SENSOR_0)) or np.any(np.equal(obstacle_direction,SENSOR_4))))
print(np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_5)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_6)))
print(np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)))
print(np.any(np.equal(obstacle_direction,SENSOR_2)))

#robot.set_var("motor.left.target", 50)
#robot.set_var("motor.right.target", 2**16-50)
#time.sleep(1.1)
robot.set_var("motor.left.target", 0)
robot.set_var("motor.right.target", 0)
              


[0, 0, 0, 0, 0, 0, 2268]
[6]
False
False
True
False
True
False


Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\roxan\Documents\Applications\Anaconda\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\roxan\Documents\Applications\Anaconda\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\roxan\Documents\EPFL_COURS\Basics of mobile robotics\Solutions_4\Solutions\src\Thymio.py", line 340, in do_refresh
    self.get_variables()
  File "C:\Users\roxan\Documents\EPFL_COURS\Basics of mobile robotics\Solutions_4\Solutions\src\Thymio.py", line 499, in get_variables
    self.send(msg)
  File "C:\Users\roxan\Documents\EPFL_COURS\Basics of mobile robotics\Solutions_4\Solutions\src\Thymio.py", line 456, in send
    self.io.write(msg.serialize())
  File "C:\Users\roxan\Documents\Applications\Anaconda\lib\site-packages\serial\serialwin32.py", line 323, in write
    raise writeTimeoutError
serial.serialutil.SerialTimeoutException: Write timeout


### Bugs to fix

1) if the robot meet a v shape obstacle  
2) if the robot has an obstacle in front of sensor n°2 it can turns and then sense the obstacle on the 3th, it turns backwards --> wait a bit more in the case of detection on the sensor 2 ?